In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
pip install opencv-python

In [4]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00


In [5]:
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.solutions import ObjectCounter


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
MODEL_PATH = "/content/drive/MyDrive/YOLO11m train dataset/best (1).pt"
VIDEO_PATH = "/content/drive/MyDrive/cctv.mp4/market.mp4"
OUTPUT_PATH = "/content/output_with_counting.mp4"

model = YOLO(MODEL_PATH)


Video setup

In [7]:
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise FileNotFoundError("Video not found")

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

writer = cv2.VideoWriter(
    OUTPUT_PATH,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (w, h)
)


Entry Exit

In [8]:
frame_idx = 0

first_seen = {}        # id frame index first seen
last_seen = {}         # id frame index last seen
counted_entry = set()
counted_exit = set()

MIN_FRAMES_TO_COUNT = 8     #  must stay visible
EXIT_TIMEOUT = 20           #  must be gone


Logic / Loop

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1

    results = model.track(
        frame,
        persist=True,
        tracker="bytetrack.yaml",
        conf=0.4,
        iou=0.5
    )

    visible_ids = set()

    # Define class_names here from the model
    class_names = model.names

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        ids = results[0].boxes.id.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()

        for box, tid, cls in zip(boxes, ids, classes):
            tid = int(tid)
            visible_ids.add(tid)

            if tid not in first_seen:
                first_seen[tid] = frame_idx

            last_seen[tid] = frame_idx

            # -------------------------
            # ENTRY (STABLE)
            # -------------------------
            if (
                tid not in counted_entry and
                frame_idx - first_seen[tid] >= MIN_FRAMES_TO_COUNT
            ):
                counted_entry.add(tid)

            # Gender (FIXED)
            gender = class_names[int(cls)] # Changed CLASS_NAMES to class_names

            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"ID {tid} {gender}",
                (x1, y1 - 8),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

    # -------------------------
    # EXIT (STABLE)
    # -------------------------
    for tid in counted_entry:
        if (
            tid not in visible_ids and
            tid not in counted_exit and
            frame_idx - last_seen.get(tid, 0) >= EXIT_TIMEOUT
        ):
            counted_exit.add(tid)

    entered = len(counted_entry)
    exited = len(counted_exit)
    current = entered - exited

    cv2.putText(
        frame,
        f"Entered: {entered}   Exited: {exited}   Current: {current}",
        (30, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        3
    )

    writer.write(frame)

Streaming output truncated to the last 5000 lines.
0: 640x480 2 males, 1738.4ms
Speed: 6.3ms preprocess, 1738.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 males, 1603.2ms
Speed: 4.8ms preprocess, 1603.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 2 males, 1103.8ms
Speed: 7.7ms preprocess, 1103.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 male, 1142.5ms
Speed: 4.5ms preprocess, 1142.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 male, 1116.6ms
Speed: 4.3ms preprocess, 1116.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 male, 1169.4ms
Speed: 4.9ms preprocess, 1169.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 male, 1134.3ms
Speed: 4.2ms preprocess, 1134.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 male, 1132.4ms
Speed: 4.9ms preproces

In [12]:
cap.release()
writer.release()
cv2.destroyAllWindows()

print("/content/output_with_counting.mp4", OUTPUT_PATH)


/content/output_with_counting.mp4 /content/output_with_counting.mp4
